# Introducció

Els bots de telegram mai poden començar una conversació. A l'inici, sempre, l'usuari ha d'enviar la comanda `/start` per començar la interacció. Però, cal tenir en compte que en qualsevol moment podem rebre aquesta comanda. 

El funcionament serà el següent:

1. Quan l'usuari enviï `/start`, aquest passarà a l'estat `ADDING`, és a dir, afegint productes a la llista. 
2. Qualsevol missatge nou que es rebi, s'assumirà que és un nou producte a afegir
3. Al rebre `/done`, passarà a estar a l'estat `FLAGGING`, és a dir marcant productes com a comprats
4. Qualsebol missatge que es rebi, indicarà que el producte (contingut del missatge) ha estat comprat
5. Al comprar-ho tot, el bot simplement dirà que la llista està buida

Es farà de forma progressiva, i no sempre tractant directament amb el bot per tal de simplificar

# 1. Persistència de sessions

Tal i com hem vist, les connexions tenen una durada limitada, i totes les dades associades a aquestes es perden. En canvi, tot usuari té sempre un identificador únic (`self.id`), que ens pot servir per mantenir, de forma externa a *telepot* un conjunt de dades persistents en memòria.

Volem aconseguir que el fluxe de la nostra aplicació sigui com el següent:
**Tingueu en compte que TOTA la part esquerra ja està feta! Solament falta la persistència, que correspon a `UserInfo`.**

<img src="seq.png" />

### 1.1. Crea una funció que donat l'identificador d'un usuari, en retorni un diccionari a mode de "base de dades"

**§1.1.1**
* Identificador d'usuari, seguint el format `id: <int>`
* Estat de l'usuari: `status: <ADDING/FLAGGING>`, per defecte `ADDING`. Corresponen als estats d'afegir productes i de marcar-los com a comprats, respectivament.
* Historial de missatges de més antic a més modern, com a llista d'strings: `messages: [<str>, <str>, ...]`
* Llista de la compra, una llista (`products: []`) on cada element és un diccionari que consisteix dels següents camps:
  * `id: <str>` és el nom del producte
  * `status: <PENDING/BOUGHT>` indica si està comprat o no
  * `qty: <int>` quants productes es vol comprar d'aquest mateix identificador
 
**§1.1.2** Dins de la llista de productes no pot existir més d'un element amb el mateix `id`. En cas de repetició, cal sumar a la quantitat del ja existent la d'aquest. Si el producte ja existent figura amb l'estat `BOUGHT`, la funció llençarà una excepció de tipus `TypeError`.
  
**§1.1.3** Si l'usuari no existeix en la base de dades, l'ha de crear i retornar seguint el format de dalt

Per exemple, el següent és un usuari vàlid del sistema

```python
{
  'id': 1234,
  'status': 'FLAGGING',
  'messages': ['/start', 'llet', 'ous 6', '/done'],
  'products': [
    {
      'id': 'llet',
      'status': 'PENDING',
      'qty': 1
    },
    {
      'id': 'ous',
      'status': 'PENDING',
      'qty': 6
    },
  ]
}
```

Un usuari nou, just creat, tindria la següent aparença

```python
{
  'id': 1234,
  'status': 'ADDING',
  'messages': [],
  'products': []
}
```

----------------------------

**IMPORTANT**: 

1. Es demana persistència en memòria RAM, **no és necessari** que les dades es guardin entre diferents sessions de bot (tancar-l'ho i obrir-l'ho).

2. La forma en que emmagatzemeu les dades és indiferent (diccionaris, llistes, DataFrames, etc.), sempre i quan retorneu amb el format apropiat! Podeu fer servir variables globals

In [1]:
import json
def user_info(user_id):
    """
    Retorna, i crea si és necessari, la informació corresponent a un
    usuari, des de que s'ha iniciat el bot fins ara. És a dir, aquesta
    informació és persistent.

    :param user_id: Identificador de l'usuari a retornar, un enter
    :return: Un diccionari amb el format especificat en l'enunciat
        superior
    """
    
    
    with open('datos.json', 'r') as f:
        data = json.load(f)
        
    '''
    data.remove({"id": 2})
    
    
    with open('datos.json','w') as f:
        json.dump(data,f)
    '''
    
    if str(user_id) in data:
        
        return(data[str(user_id)])
    
    else:
        
        
        data[str(user_id)] = {
                                "id": user_id,
                                "status": "FLAGGING",
                                "messages": [],
                                "products": []
                            }
        with open('datos.json','w') as f:
            json.dump(data,f,indent= 4)
        
        return (data[str(user_id)])
        
    
    
    

### 1.2. Crea tres funcions que permetin modificar les dades de l'usuari

  1. **Afegir missatges a l'historial de l'usuari**
  2. **Afegir productes**, tenint en compte possibles repeticions, a la llista de productes
  3. **Marcar un producte** com a `PENDING`/`BOUGHT`. Si s'intenta marcar un producte inexistent o bé del mateix estat en el que ja està, llença una excepció `TypeError`.

In [2]:
def add_message(user_id, msg):
    """
    Donat un identificador d'un usuari i un missatge (text),
    s'afegeix aquest missatge a la informació persitent de
    l'usuari
    
    :param user_id: Identificador de l'usuari, un enter
    :param msg: Missatge a afegir
    """
    
    with open('datos.json', 'r') as f:
        data = json.load(f)
    
    data[str(user_id)]["messages"].append(msg)
    
    
    with open('datos.json','w') as f:
        json.dump(data,f,indent = 4)
    
    
    
def add_product(user_id, prod_id, qty):
    """
    Donat un identificador d'un usuari, un producte i una
    quantitat, s'afegeix a la llista de productes de
    l'usuari
    
    :param user_id: Identificador de l'usuari, un enter
    :param prod_id: Identificador del producte, una string
    :param qty: Quantitat del producte, un enter
    :raise TypeError: Si el producte que s'intenta afegir
        ja està a la llista de productes i, a més, consta
        com a ja comprat, la funció llença un error
    """
    
    
    
    with open('datos.json', 'r') as f:
        data = json.load(f)
        
    
    flag = False
    for i in data[str(user_id)]["products"]:
        if i["id"] == prod_id:
            flag = True
            break
            
    if not flag:
         data[str(user_id)]["products"].append({
                                  'id': prod_id,
                                  'status': 'PENDING',
                                  'qty': qty
                                })
            
    
    with open('datos.json','w') as f:
        json.dump(data,f,indent = 4)
    
    
    
    
def flag_product(user_id, prod_id, status):
    """
    Donat un identificador d'un usuari, un producte
    i un estat al que es vol canviar, la funció en
    canvia l'estat si pot (llegir :raise TypeError:)
    
    :param user_id: Identificador de l'usuari, un enter
    :param prod_id: Identificador del producte, una string
    :param status: Estat al que es vol canviar, una string
    :raise TypeError: Si el producte (prod_id) no està a la llista
        de producte de l'usuari, es llença un error
    :raise TypeError: Si el producte està a la llista però ja té
        l'estat al que s'intenta canviar, es llença un error
    """
    
    
    with open('datos.json', 'r') as f:
        data = json.load(f)
        
    flag = False
    for i in data[str(user_id)]["products"]:
        if i["id"] == prod_id and i["status"] != status:
            flag = True
            i["status"] = status
            break
        
        
    with open('datos.json','w') as f:
        json.dump(data,f,indent = 4)
        
    
    if not flag:
        return False
    return True

### 1.3. Crea una funció que netegi per complet la informació guardada d'un usuari

Simplement, deixa el diccionari corresponent a l'usuari com si acabés de crear-se, sense absolutament cap informació.

In [3]:
def clear_info(user_id):
    """
    Borra tota la informació, o la deixa com a l'estat
    inicial, d'un usuari
    
    :param user_id: Identificador de l'usuari, un enter
    """
    
    with open('datos.json', 'r') as f:
        data = json.load(f)
        
    if str(user_id) in data:
        
         data[str(user_id)] = {
                                "id": user_id,
                                "status": "FLAGGING",
                                "messages": [],
                                "products": []
                            }
    with open('datos.json','w') as f:
        json.dump(data,f,indent = 4)

### Conjunt de proves per tot 1)

Podeu modificar-lo com cregueu adient sense cap mena de problema

In [4]:
if __name__ == '__main__':
    pass

# 2. Creant el bot

**§2.1** Sempre que rebem un `/start`, el bot ha de borrar qualsevol informació de persistència de l'usuari, per tal de començar nets. A més, respondrà a l'usuari amb un missatge de benvinguda. Per simplificar, quan el bot rebi la comanda `/start`, començarà a apuntar qualsevol missatge que l'usuari enviï, sense incloure el propi `/start`. 

**§2.2** A més, cada missatge per separat indica un nou producte que es vol afegir. 

**§2.3** Si rep la comanda `/done` haurà de deixar d'apuntar nous productes a la llista, però no d'afegir a l'historial de missatges!

**§2.4** El bot no ha de respondre res mentre l'usuari entri productes.

**BONUS**: Podeu obtenir punts extra si feu que el bot entengui missatges del tipus: `<nom del producte> <quantitat>` o `<quantitat> <nom del producte>`, com per exemple `ous 6`, `6 ous`, `ous de guatlla 6` i `6 ous de guatlla`.

**§2.5** Al rebre `/done`, el bot enviarà la llista completa al client. Tota la llista serà 1 sol missatge, on cada línia serà un producte en el format `<id>: <qty>`. 

**§2.6** Qualsevol missatge rebut a continuació (excepte `/start`, és clar) indicarà que un producte ha estat comprat, i s'haurà de respondre amb la llista dels productes restants. En cas de `TypeError`, envia un missatge explicant que ha passat.

**§2.7** L'usuari pot enviar en qualsevol moment la comanda `/list`. Al fer-ho, el bot enviarà la llista d'items que encara no han estat comprats a l'usuari. En cas de tenir una llista buida, respondrà `No hi ha productes`.

Per tant, recapitulant, has de fer:

* Al rebre la comanda `/start`, netejar tota la informació de l'usuari
* Grabar qualsevol missatge de l'usuari, inclosos `/start` i `/done` i posteriors
* Entre `/start` i `/done`, comptar cada missatge excepte les comandes com a un producte a afegir a la llista
* Després de `/done`, els missatges han de cambiar l'estat del producte a `BOUGHT`.

**BONUS**: A Telegram, es poden fer edicions del text enviat prèviament. Fes que el bot, cada cop que cambia un producte a BOUGHT, editi el missatge de la llista de productes i tatxi el producte en qüestió.

**BONUS**: L'usuari a més d'enviar text pot respondre amb `callbacks` i botons "prefabricats" pel bot. De manera que es pot limitar quines respotes pot donar l'usuari (evitant per exemple que "compri" productes que no estan a la llista). Investiga alguna de les solucions que s'ofereixen en aquest sentit i integra-la en el teu bot.

## Respondre missatges

Des de dins la funció `on_chat_message`, es pot respondre a l'usuari mitjançant la crida `await self.sender.sendMessage('missatge')`. Es pot cridar tants cops com vulgueu, cada crida correspondrà a un missatge diferent que l'usuari rebrà, en el mateix ordre que les feu.

# Avaluació

**§2.8** Per tal de poder provar que el bot funciona correctament, caldrà afegir una nova comanda a l'usuari, anomenada `/debug`, que retorni la informació de l'usuari (com a JSON serialitzat, en text).

Es pot rebre `/debug` en qualsevol moment de l'execució. Aquest ha de quedar enregistrat en l'historial de missatges, però no ha de ser tractat com a un producte.

**Nota**: Aquesta comanda, o d'altres amb funcionalitats semblants, mai les tindrieu en un bot real! És solament per propòsits d'avaluació de la pràctica.

In [5]:
import asyncio as aio

import telepot
from telepot.aio.loop import MessageLoop
from telepot.aio.delegate import pave_event_space, per_chat_id, \
    create_open, include_callback_query_chat_id


class ShoppingBot(object):
    """
    Classe principal del bot, configura els clients i inicia
    el bucle per rebre i enviar missatges
    """
    
    def __init__(self):
        """
        Constructor de la classe, inicialitza el bot
        """
        self.bot = None
        self.loop = aio.get_event_loop()
        
    def start(self, token):
        """
        Inicia el bucle per rebre i enviar missatges. Bloqueja per
        complet fins que no s'acaba d'executar
        
        :param token: Token del bot per conenctar a l'API de telegram
        """
        self.bot = telepot.aio.DelegatorBot(token, [
            include_callback_query_chat_id(
                pave_event_space())(
                per_chat_id(), create_open, ShoppingUser, timeout=10),
            ])
        
        self.loop.create_task(MessageLoop(self.bot).run_forever())
        self.loop.run_forever()

In [6]:
class ShoppingUser(telepot.aio.helper.ChatHandler):
    """
    Classe per instanciar cada usuari i gestionar-ne els
    missatges.
    """
    
    def __init__(self, *args, **kwargs):
        """
        Constructor de la classe, amb l'únic fi de mostrar un missatge
        per pantalla indicant que s'ha creat l'usuari
        """
        super(ShoppingUser, self).__init__(*args, **kwargs)
        user_info(self.id)
        print('Created {}'.format(self.id))
        self.status = 0
        self.listaProd = []
        
        
    async def on_chat_message(self, msg):
        """
        Funció que Telepot cridarà de forma automàtica quan un usuari
        enviï un missatge
        
        :param msg: Objecte que conté, d'entre altres, el missatge que
            l'usuari ha enviat
        """
        content_type, chat_type, chat_id = telepot.glance(msg)
        if 'text' in msg:
            
            
            
            if msg['text'] == "/done" and self.status == 1:
                self.status= 0
                
                for i in self.listaProd:
                    await self.sender.sendMessage(i)
            
            elif msg['text'] == "/debug":
                await self.sender.sendMessage(user_info(self.id))
                add_message(self.id,msg['text'])
                
                
                
            elif msg['text'] == "/list":
                with open('datos.json', 'r') as f:
                    data = json.load(f)
                listP = []
                vacia = True

                for i in data[str(self.id)]["products"]:
                    if i["status"] != "Comprado":
                        listP.append(str(i["qty"]) + " - " + str(i["id"]))
                        vacia = False
                if not vacia:
                    await self.sender.sendMessage("Productos pendientes: ")
                    for i in listP:
                        await self.sender.sendMessage(i)
                else:
                    await self.sender.sendMessage("No hay más productos por comprar")
                
            elif self.status == 1 and msg['text'] != "/done":
                txtSplit = msg['text'].split(" ")
                cant = 0
                idind = ""
                
                if len(txtSplit) == 1:
                    idind = txtSplit.pop()
                    add_product(self.id,idind , 1)
                    self.listaProd.append(str(1) + " - " + idind)
                    
                else:
                    try:
                        cant = int(txtSplit.pop(0))
                        idind =txtSplit.pop()
                    except :
                        try:
                            txtSplit = msg['text'].split(" ")
                            cant = int(txtSplit.pop())
                            txtSplit.pop()
                            idind = txtSplit.pop()
                        except :
                            print("Error")


                    if cant != 0 and cant>0:

                        add_product(self.id, idind, cant)
                        self.listaProd.append(str(cant) + " - " + idind)

                
                  
                
                add_message(self.id,msg['text'])
                
            elif msg['text'] == "/start" and self.status != 1:
                self.status = 1
                self.listaProd = []
                
                
                clear_info(self.id)
                await self.sender.sendMessage("bienvenido: " + str(self.id))
                
            else: 
                if (flag_product(self.id, msg['text'], "Comprado")):
                    
                    with open('datos.json', 'r') as f:
                        data = json.load(f)
                    listP = []
                    vacia = True

                    for i in data[str(self.id)]["products"]:
                        if i["status"] != "Comprado":
                            listP.append(str(i["qty"]) + " - " + str(i["id"]))
                            vacia = False
                    if not vacia:
                        await self.sender.sendMessage("Productos pendientes: ")
                        for i in listP:
                            await self.sender.sendMessage(i)
                    else:
                        await self.sender.sendMessage("No hay más productos por comprar")
                else:
                    await self.sender.sendMessage("El producto no esta en tu lista")
                

                
                
                add_message(self.id,msg['text'])
        
        print('From {}: {}'.format(self.id,msg['text'] ))
                
            
    async def on_close(self, ex):
        """
        Passats 10 segons (o els configurats) d'inactivitat de l'usuari,
        aquest s'elimina de la memòria. Abans però, Telepot crida 
        automàticament aquesta funció per informar-nos i, si cal, poder
        fer quelcom
        
        :param ex: Motiu pel qual es tanca l'usuari, normalment timeout
        """
        print('Closed {}'.format(self.id))

In [7]:
if __name__ == '__main__':
    bot = ShoppingBot()
    bot.start(open('TOKEN').read().strip())

KeyboardInterrupt: 